In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import pickle
# pd.set_option('display.max_columns', None)
# # pd.set_option("display.max_rows", 50)

import warnings

# Use this filter to ignore the specific FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

target_folder_path = '/Volumes/Database/Original_data'
%cd "{target_folder_path}"

/Volumes/Database/Original_data


### Get correct_datetime_df "open"

In [3]:
open = pd.read_csv("/Volumes/Database/Factor_data/open.csv", index_col=0)
open

,000116,000930,000960,0009A0,1101,1102,1103,1104,1107,1108,...,9945,9946,9949,9950,9951,9955,9958,9960,9962,L2301
datetime,,,,,,,,,,,,,,,,,,,,,
1980-01-04,NaN,NaN,NaN,NaN,24.90,18.80,33.40,18.50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-05,NaN,NaN,NaN,NaN,25.60,19.40,33.40,18.60,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-07,NaN,NaN,NaN,NaN,25.70,19.40,33.40,18.80,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-08,NaN,NaN,NaN,NaN,26.00,20.00,33.40,18.80,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-09,NaN,NaN,NaN,NaN,25.90,19.80,33.40,18.70,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-06,NaN,NaN,NaN,NaN,32.00,40.25,17.55,31.00,NaN,16.20,...,35.65,24.85,17.60,20.10,72.8,24.85,249.0,27.00,17.9,NaN
2024-03-07,NaN,NaN,NaN,NaN,32.00,40.00,17.55,31.20,NaN,16.25,...,36.10,25.25,18.00,21.70,73.1,25.05,248.0,27.15,17.8,NaN
2024-03-08,NaN,NaN,NaN,NaN,32.00,40.30,17.40,31.10,NaN,15.90,...,35.80,25.40,17.65,21.05,72.1,25.50,242.0,27.20,17.6,NaN


In [3]:
FR_data = pd.read_table(rf"{target_folder_path}/IFRS合併一般產業單季/財報.txt", encoding='Big5-HKSCS', low_memory=False)

In [4]:
columns_list = FR_data.columns.tolist()
the_place_start_to_become_numeric = 8

### make all_columns to str
for column in columns_list:
    FR_data[column] = FR_data[column].astype(str).str.strip()

### make numeric data to be numeric
numeric_list = [element for element in columns_list[the_place_start_to_become_numeric:] if "發布日" not in element] # [2:] is become I need to make the future symbol columns "ticker" keep in str
for elements in numeric_list:
    FR_data[elements] = pd.to_numeric(FR_data[elements], errors='coerce')

### make date inf to datetime
FR_data['年/月'] = pd.to_datetime(FR_data['年/月'], format='%Y%m')
elements_with_releseday = [element for element in columns_list[the_place_start_to_become_numeric:] if "發布日" in element]
for elements in elements_with_releseday:
    FR_data[elements] = pd.to_datetime(FR_data[elements], format='%Y/%m/%d')

FR_data.rename(columns= {'公司': 'symbol',
                         '年/月': 'datetime',
                         '財報發布日': 'release'}, inplace=True)

In [5]:
FR_data.columns = FR_data.columns.str.replace('/', '')
output_list = [col for col in FR_data.columns[the_place_start_to_become_numeric:] if col not in elements_with_releseday]

In [6]:
# # 100% according to publish date (date rebalance)
# for column_name in output_list:
#     print(column_name)
#     unstackdata_release = FR_data.set_index(['datetime','symbol'])['release'].unstack().loc['2013':]
#     unstackdata_imf = FR_data.set_index(['datetime','symbol'])[column_name].unstack().loc['2013':]
    
#     unstackdata_release = unstackdata_release.reindex(open.index).fillna(method='ffill')
#     filter = unstackdata_release.apply(lambda x: x == x.index)
#     unstackdata_imf = unstackdata_imf.reindex(open.index).fillna(method='ffill')
#     unstackdata_imf_filtered = unstackdata_imf[filter]
#     data_reindex = unstackdata_imf_filtered.fillna(method='ffill', inplace=False)
#     csv_file = os.path.join('/Volumes/Database/Factor_data', f'{column_name}_100.csv')
#     data_reindex.to_csv(csv_file)
        

In [33]:
# 90% first release (quarter rebalance)
unstackdata_release = FR_data.reset_index().set_index(['datetime','symbol'])['release'].unstack().loc['2013':]
open_indexes = open.loc['2013':].index

for column_name in output_list:
    unstackdata_imf = FR_data.reset_index().set_index(['datetime','symbol'])[column_name].unstack().loc['2013':]
    releaserankpct = unstackdata_release.rank(axis=1,pct=True)
    filter = unstackdata_imf[releaserankpct < 0.9]
    filter.index = unstackdata_release[releaserankpct < 0.9].max(axis=1)
    data_reindex = filter.reindex(open_indexes, method = 'ffill') # 不同於fillna(ffill) 對NaN的處理不同
    csv_file = os.path.join('/Volumes/Database/Factor_data', f'{column_name}_90.csv')
    data_reindex.to_csv(csv_file)


### check